# Runnning experiments on the Adult dataset

## Installation

First we need to install EthicML. You can install it from PyPI:

```
pip install ethicml
```

## Loading the data

EthicML includes some often used datasets from fairness literature.
First, we load one of these... in this example we load the UCI Adult dataset

In [1]:
import ethicml as em
from ethicml.data import Adult

adult = Adult()
data: em.DataTuple = adult.load()
assert (45222, 101) == data.x.shape
assert (45222,) == data.s.shape
assert (45222,) == data.y.shape

This loads the dataset as a DataTuple, which comprises $x$ (features), $s$ (sensitive attribute and $y$ (class label). Each member of the DataTuple is stored as a Pandas DataFrame.

By default, the Adult dataset uses the binary attribute `sex_Male` as the sensitive feature.

In [2]:
data.s.to_frame().head()

,sex_Male
0,1
1,1
2,1
3,1
4,0


If we want to run experiments using race as the sensitive attribute we could change that manually, or, as this is a common task, EthicML can split the data for you.

In [3]:
data: em.DataTuple = Adult(split=Adult.Splits.RACE).load()
assert (45222, 98) == data.x.shape
assert (45222,) == data.s.shape
assert (45222,) == data.y.shape

In [4]:
data.s.to_frame().head()

,race
0,4
1,4
2,4
3,2
4,4


However, we're going to be repeating some of the experiments from FairGP. In that paper they do experiments with race as the sensitive attribute, but the value is binary. The value of race is White or Not_White.

Fortunately, EthicML has a split for that: `RACE_BINARY`.

In [5]:
data = Adult(split=Adult.Splits.RACE_BINARY).load()

In [6]:
data.s.to_frame().head()

,race_White
0,1
1,1
2,1
3,0
4,1


## Evaluating some models

In [7]:
from ethicml.run import evaluate_models
from ethicml.models import LR, Reweighting, SVM, Upsampler
from ethicml.metrics import Accuracy, CV, ProbPos, TPR

results = evaluate_models(
    datasets=[em.data.Adult(split=Adult.Splits.RACE_BINARY)],
    preprocess_models=[Upsampler()],
    inprocess_models=[LR(), SVM(kernel=em.KernelType.linear), Reweighting()],
    metrics=[Accuracy(), CV()],
    per_sens_metrics=[Accuracy(), TPR(), ProbPos()],
    repeats=2,
)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.5s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    1.6s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    2.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    2.4s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed:    2.7s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed:    2.7s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.3s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.4s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.4s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with

In [8]:
results

model_seed  \
dataset           scaler transform        model                       split_id               
Adult Race-Binary None   no_transform     Logistic Regression (C=1.0) 0                0.0   
                                          SVM (linear)                0                0.0   
                                          Kamiran & Calders lr C=1.0  0                0.0   
                                          Logistic Regression (C=1.0) 1                1.0   
                                          SVM (linear)                1                1.0   
                                          Kamiran & Calders lr C=1.0  1                1.0   
                         Upsample uniform Logistic Regression (C=1.0) 0                0.0   
                                          SVM (linear)                0                0.0   
                                          Kamiran & Calders lr C=1.0  0                0.0   
                                          Logistic Regression (C=1.0) 1                0.0   
                                          SVM (linear)                1                0.0   
                                          Kamiran & Calders lr C=1.0  1                0.0   

                                                                                  seed  \
dataset           scaler transform        model                       split_id           
Adult Race-Binary None   no_transform     Logistic Regression (C=1.0) 0            0.0   
                                          SVM (linear)                0            0.0   
                                          Kamiran & Calders lr C=1.0  0            0.0   
                                          Logistic Regression (C=1.0) 1         2410.0   
                                          SVM (linear)                1         2410.0   
                                          Kamiran & Calders lr C=1.0  1         2410.0   
                         Upsample uniform Logistic Regression (C=1.0) 0            0.0   
                                          SVM (linear)                0            0.0   
                                          Kamiran & Calders lr C=1.0  0            0.0   
                                          Logistic Regression (C=1.0) 1         2410.0   
                                          SVM (linear)                1         2410.0   
                                          Kamiran & Calders lr C=1.0  1         2410.0   

                                                                                  C  \
dataset           scaler transform        model                       split_id        
Adult Race-Binary None   no_transform     Logistic Regression (C=1.0) 0         1.0   
                                          SVM (linear)                0         1.0   
                                          Kamiran & Calders lr C=1.0  0         NaN   
                                          Logistic Regression (C=1.0) 1         1.0   
                                          SVM (linear)                1         1.0   
                                          Kamiran & Calders lr C=1.0  1         NaN   
                         Upsample uniform Logistic Regression (C=1.0) 0         1.0   
                                          SVM (linear)                0         1.0   
                                          Kamiran & Calders lr C=1.0  0         NaN   
                                          Logistic Regression (C=1.0) 1         1.0   
                                          SVM (linear)                1         1.0   
                                          Kamiran & Calders lr C=1.0  1         NaN   

                                                                                Accuracy  \
dataset           scaler transform        model                       split_id             
Adult Race-Binary None   no_transform     Logistic Regression (C=1.0) 0         0.851078   
       

In [9]:
results[
    ['Accuracy', 'Accuracy_race_White_0÷race_White_1', 'TPR_race_White_0÷race_White_1', 'prob_pos_race_White_0÷race_White_1']
].groupby(level=[0, 1, 2]).agg(['mean', 'std'])

Accuracy            \
                                               mean       std   
dataset           scaler transform                              
Adult Race-Binary None   Upsample uniform  0.845955  0.004519   
                         no_transform      0.847172  0.004514   

                                          Accuracy_race_White_0÷race_White_1  \
                                                                        mean   
dataset           scaler transform                                             
Adult Race-Binary None   Upsample uniform                           0.941926   
                         no_transform                               0.941692   

                                                     \
                                                std   
dataset           scaler transform                    
Adult Race-Binary None   Upsample uniform  0.002026   
                         no_transform      0.002408   

                                          TPR_race_White_0÷race_White_1  \
                                                                   mean   
dataset           scaler transform                                        
Adult Race-Binary None   Upsample uniform                      0.985585   
                         no_transform                          0.978839   

                                                     \
                                                std   
dataset           scaler transform                    
Adult Race-Binary None   Upsample uniform  0.019223   
                         no_transform      0.013463   

                                          prob_pos_race_White_0÷race_White_1  \
                                                                        mean   
dataset           scaler transform                                             
Adult Race-Binary None   Upsample uniform                           0.678826   
                         no_transform                               0.655388   

                                                     
                                                std  
dataset           scaler transform                   
Adult Race-Binary None   Upsample uniform  0.026941  
                         no_transform      0.024718